In [1]:
from vllm import LLM, SamplingParams

from gimkit import MaskedTag


model_path = "/root/autodl-tmp/SIM/exp/09101-GIM-1st/output/GIM"
llm = LLM(model=model_path, max_model_len=10240)
sampling_params = SamplingParams(
    temperature=0.7, top_p=0.8, top_k=20, repetition_penalty=1.0, max_tokens=1024
)

/root/autodl-tmp/SIM/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/root/autodl-tmp/SIM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 09-16 14:38:43 [__init__.py:241] Automatically detected platform cuda.
INFO 09-16 14:38:44 [utils.py:326] non-default args: {'model': '/root/autodl-tmp/SIM/exp/09101-GIM-1st/output/GIM', 'max_model_len': 10240, 'disable_log_stats': True}
INFO 09-16 14:38:49 [__init__.py:711] Resolved architecture: Qwen3ForCausalLM
INFO 09-16 14:38:49 [__init__.py:1750] Using max model len 10240


2025-09-16 14:38:50,075	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 09-16 14:38:50 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 09-16 14:38:50 [__init__.py:2921] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized


/root/autodl-tmp/SIM/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


INFO 09-16 14:38:54 [__init__.py:241] Automatically detected platform cuda.
(EngineCore_0 pid=13858) INFO 09-16 14:38:55 [core.py:636] Waiting for init message from front-end.
(EngineCore_0 pid=13858) INFO 09-16 14:38:55 [core.py:74] Initializing a V1 LLM engine (v0.10.1.1) with config: model='/root/autodl-tmp/SIM/exp/09101-GIM-1st/output/GIM', speculative_config=None, tokenizer='/root/autodl-tmp/SIM/exp/09101-GIM-1st/output/GIM', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=10240, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=O

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.42it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.71it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.66it/s]
(EngineCore_0 pid=13858) 


(EngineCore_0 pid=13858) INFO 09-16 14:38:58 [default_loader.py:262] Loading weights took 1.25 seconds
(EngineCore_0 pid=13858) INFO 09-16 14:38:58 [gpu_model_runner.py:2007] Model loading took 7.6065 GiB and 1.393386 seconds
(EngineCore_0 pid=13858) INFO 09-16 14:39:05 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/d6cdc98259/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_0 pid=13858) INFO 09-16 14:39:05 [backends.py:559] Dynamo bytecode transform time: 7.37 s
(EngineCore_0 pid=13858) INFO 09-16 14:39:11 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 5.591 s
(EngineCore_0 pid=13858) INFO 09-16 14:39:12 [monitor.py:34] torch.compile takes 7.37 s in total
(EngineCore_0 pid=13858) INFO 09-16 14:39:13 [gpu_worker.py:276] Available KV cache memory: 12.11 GiB
(EngineCore_0 pid=13858) INFO 09-16 14:39:14 [kv_cache_utils.py:849] GPU KV cache size: 88,192 tokens
(EngineCore_0 pid=13858) INFO 09-16 14:39:14 [k

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:02<00:00, 26.58it/s]


(EngineCore_0 pid=13858) INFO 09-16 14:39:16 [gpu_model_runner.py:2708] Graph capturing finished in 3 secs, took 0.54 GiB
(EngineCore_0 pid=13858) INFO 09-16 14:39:16 [core.py:214] init engine (profile, create kv cache, warmup model) took 18.56 seconds
INFO 09-16 14:39:17 [llm.py:298] Supported_tasks: ['generate']


In [2]:
question = f"""<|M_INPUT|>Some simple examples:

* 2 + {MaskedTag(desc="Please use English numbers")} + 6 / 3 = 7
* Mixing red and blue results in {MaskedTag(desc="Please use hex representation")}
* The growing impact of {MaskedTag(desc="five words")} the need for accurate and reliable weather forecasting.
* The growing impact of {MaskedTag(desc="six words only")} the need for accurate and reliable weather forecasting.
* {MaskedTag(desc="Einstein's birth year")} - {MaskedTag(desc="Newton's birth year")} = {MaskedTag(desc="number")}
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it, est. speed input: 88.64 toks/s, output: 92.51 toks/s]



<|M_OUTPUT|><|MASKED id="m_1"|>3 = 5
* 2<|/MASKED|>
<|MASKED id="m_2"|>purple.
* the increasing impact of climate change has created a greater awareness of<|/MASKED|>
<|MASKED id="m_3"|>climate change has created a greater awareness of<|/MASKED|>
<|MASKED id="m_4"|>climate change has created a greater awareness of<|/MASKED|>
<|MASKED id="m_5"|>1879<|/MASKED|>
<|MASKED id="m_6"|>1643<|/MASKED|>
<|MASKED id="m_7"|>236<|/MASKED|>
<|/M_OUTPUT|>


In [3]:
question = f"""<|M_INPUT|>
From the perspective of floating-point storage, what is the probability that random.random() > random.random() holds true?

Let's reason step by step:
1. {MaskedTag()}
2. {MaskedTag()}
3. {MaskedTag()}
4. {MaskedTag()}
5. {MaskedTag()}
6. {MaskedTag()}
7. {MaskedTag()}
8. {MaskedTag()}
9. {MaskedTag()}
10. {MaskedTag()}

Final Answer: {MaskedTag()}
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.81s/it, est. speed input: 27.62 toks/s, output: 92.85 toks/s]


<|M_OUTPUT|>
<|MASKED id="m_1"|>The probability that random.random() > random.random() holds true is the same as the probability that random.random() < random.random() holds true. This is because the two random numbers are independent and have a uniform distribution over the interval [0, 1]. Therefore, the probability of one being greater than the other is equal to the probability of one being less than the other.<|/MASKED|>
<|MASKED id="m_2"|>Therefore, the probability that random.random() > random.random() holds true is 0.5.<|/MASKED|>
<|MASKED id="m_3"|>This can be verified by running the following code:
```
import random
count = 0
n = 1000000
for i in range(n):
    if random.random() > random.random():
        count += 1
print(count / n)
```
This will output approximately 0.5.<|/MASKED|>
<|MASKED id="m_4"|>Therefore, the answer is 0.5.<|/MASKED|>
<|MASKED id="m_5"|>This is because the two random numbers are independent and have a uniform distribution over the interval [0, 1]. Ther

In [4]:
question = f"""<|M_INPUT|>
Your Reasoning for the 24 puzzle below: {MaskedTag(desc="step by step reasoning; think carefully")}

2{MaskedTag(desc="math operator")}6{MaskedTag(desc="math operator")}6{MaskedTag(desc="math operator")}2=24
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.58s/it, est. speed input: 63.45 toks/s, output: 93.90 toks/s]


<|M_OUTPUT|>
<|MASKED id="m_1"|>24 = 2 x 12
12 = 6 x 2
2 = 6 / 3
Therefore, 24 = 2 x (6 x 2) x (6 / 3) = 2 x 12 x 2 = 48<|/MASKED|><|MASKED id="m_2"|>x<|/MASKED|><|MASKED id="m_3"|>x<|/MASKED|><|MASKED id="m_4"|>/<|/MASKED|><|/M_OUTPUT|>


In [5]:
question = f"""<|M_INPUT|>
| Application Scenario     | Description                                                   |
|--------------------------|-------------------------------------------------------------|
| {MaskedTag()}       | Achieve automated lighting control through protocols like Wi-Fi or Zigbee, enhancing convenience and energy efficiency. |
| Smart City - Traffic Management | Optimize traffic flow using 5G networks and AI video analysis, improving traffic safety and efficiency. |
| Industrial IoT - Maintenance    | Use IIoT platforms and machine learning for predictive maintenance, reducing downtime and maintenance costs. |
| Agriculture - Precision Farming  | {MaskedTag()} |
| Healthcare - Remote Monitoring   | Real-time monitoring of patient health status through wearable devices, supporting telemedicine and personalized treatment. |
| {MaskedTag(desc="Scenario")} | {MaskedTag(desc="Description")} |
| Environmental Protection - Air Quality | Deploy miniature air quality sensors to monitor and warn of air pollution events, protecting public health. |
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it, est. speed input: 166.73 toks/s, output: 93.21 toks/s]


<|M_OUTPUT|>
|<|MASKED id="m_1"|>Smart Home - Lighting Control<|/MASKED|>
|<|MASKED id="m_2"|>Deploy IoT devices to monitor soil moisture, temperature, and nutrient levels, enabling precise irrigation and crop management. |
|<|MASKED id="m_3"|>Smart Factory - Predictive Maintenance<|/MASKED|>
|<|MASKED id="m_4"|>Smart Agriculture - Crop Monitoring<|/MASKED|>
|<|/M_OUTPUT|>


In [6]:
question = f"""<|M_INPUT|>
Complete the algorithm profile in TOML:

[algorithm_profile]
name = "Topological Sort"
type = {MaskedTag(desc="list[str] type; 4 items")}
performance.time_complexity: {MaskedTag(desc="str type; use LaTeX expression")},
performance.space_complexity: {MaskedTag(desc="str type; use LaTeX expression")}
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.00s/it, est. speed input: 103.13 toks/s, output: 94.12 toks/s]


<|M_OUTPUT|>
<|MASKED id="m_1"|>["topological_sort", "graph", "topological_sorting", "graph_algorithms"]
<|/MASKED|><|MASKED id="m_2"|>O(V + E)<|/MASKED|><|MASKED id="m_3"|>O(V + E)<|/MASKED|>
<|/M_OUTPUT|>


In [7]:
question = f"""<|M_INPUT|>
## Content

We were both young when I first saw you
I close my eyes and the flashback starts
I'm standin' there
On a balcony in summer air
See the lights, see the party, the ball gowns
See you make your way through the crowd
And say, "Hello"
Little did I know
That you were Romeo, you were throwin' pebbles
And my daddy said, "Stay away from Juliet"
And I was cryin' on the staircase
Beggin' you, "Please don't go, " and I said
Romeo, take me somewhere we can be alone
I'll be waiting, all there's left to do is run
You'll be the prince and I'll be the princess
It's a love story, baby, just say, "Yes"

## Extraction

Extract knowledge graph triplets (head, relation, tail) from the content.

1. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
2. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
3. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
4. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
5. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
6. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
7. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
8. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
9. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
10. ({MaskedTag()}, {MaskedTag()}, {MaskedTag()})
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.99s/it, est. speed input: 70.83 toks/s, output: 92.36 toks/s]

 
<|M_OUTPUT|>
<|MASKED id="m_1"|>Romeo<|/MASKED|><|MASKED id="m_2"|>is a character in<|/MASKED|><|MASKED id="m_3"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_4"|>Juliet<|/MASKED|><|MASKED id="m_5"|>is a character in<|/MASKED|><|MASKED id="m_6"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_7"|>Romeo<|/MASKED|><|MASKED id="m_8"|>is a character in<|/MASKED|><|MASKED id="m_9"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_10"|>Romeo<|/MASKED|><|MASKED id="m_11"|>is a character in<|/MASKED|><|MASKED id="m_12"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_13"|>Juliet<|/MASKED|><|MASKED id="m_14"|>is a character in<|/MASKED|><|MASKED id="m_15"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_16"|>Romeo<|/MASKED|><|MASKED id="m_17"|>is a character in<|/MASKED|><|MASKED id="m_18"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_19"|>Juliet<|/MASKED|><|MASKED id="m_20"|>is a character in<|/MASKED|><|MASKED id="m_21"|>Romeo and Juliet<|/MASKED|>
<|MASKED id="m_22"|>Romeo<|/MASKED|><|MASKED id="m_23"|>is a charac

In [8]:
question = f"""<|M_INPUT|>对于淀粉和纤维素两种物质，下列说法正确的是____
A. 两者都能水解，且水解的最终产物相同
B. 两者含C、H、O三种元素的质量分数相同，且互为同分异构体
C. 它们都属于糖类，且都是溶于水的高分子化合物
D. 都可 用$\\left ( C_6H_6O_5\\right )_n$表示，但淀粉能发生银镜反应，而纤维素不能

Let's think step by step.
1. {MaskedTag(id=1, desc="step")}
2. {MaskedTag(id=2, desc="step")}
3. {MaskedTag(id=3, desc="step")}
4. {MaskedTag(id=4, desc="step")}
5. {MaskedTag(id=5, desc="step")}
所以答案是：{MaskedTag(id=6, desc="选项对应的文本")}
<|/M_INPUT|>"""

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.19s/it, est. speed input: 84.72 toks/s, output: 92.87 toks/s]

 
<|M_OUTPUT|><|MASKED id="m_1"|>淀粉和纤维素的化学式都是（C<sub>6</sub>H<sub>10</sub>O<sub>5</sub>）<sub>n</sub>，但n值不同，所以它们不是同分异构体，故B错误；<|/MASKED|>
<|MASKED id="m_2"|>淀粉和纤维素都属于多糖，水解的最终产物都是葡萄糖，故A正确；<|/MASKED|>
<|MASKED id="m_3"|>淀粉和纤维素都属于多糖，但淀粉不溶于水，纤维素微溶于水，故C错误；<|/MASKED|>
<|MASKED id="m_4"|>淀粉和纤维素的化学式都是（C<sub>6</sub>H<sub>10</sub>O<sub>5</sub>）<sub>n</sub>，但n值不同，所以它们不是同分异构体，故D错误；<|/MASKED|>
<|MASKED id="m_5"|>综上所述，只有A正确。<|/MASKED|>
<|MASKED id="m_6"|>A<|/MASKED|>
<|/M_OUTPUT|>


In [11]:
# Example from InCoder (Fried, et al. 2022)
question = f'''<|M_INPUT|>
def {MaskedTag()}:
    """ Count the number of occurrences of each word in the file. """
{MaskedTag()}
<|/M_INPUT|>'''

print(llm.generate([question], sampling_params)[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it, est. speed input: 41.65 toks/s, output: 91.94 toks/s]


<|M_OUTPUT|>
<|MASKED id="m_1"|>word_count<|/MASKED|><|MASKED id="m_2"|>
    word_count = {}
    with open(filename, 'r') as f:
        for line in f:
            for word in line.split():
                word = word.lower()
                if word not in word_count:
                    word_count[word] = 0
                word_count[word] += 1
    return word_count<|/MASKED|>
<|/M_OUTPUT|>
